
### Let's time travel to May 17, 1642...


The city of Montreal was just built to accommodate the growing population, the people of the city have not yet received any public facilities such as hospitals, schools, etc. In response to the need of the people, the government decides to prioritize construction of hospitals. 

When comparing the locations for developing new medical facilities, they decide to take into consideration the level of medical assistance required per age categories,the feasibility of access, the number of patients visiting a hospital, and the expected demand. In this project, we will develop a location optimization model to integrate these key factors and help decision-makers identify the most strategic location to build the hospital(s). 

The model was designed to operate at the outward level of a postal code, where the population is grouped per outward code. The objective of the hospital location optimization model is to build facilities in high demand markets to optimize population’s access to this service (maximization problem). The demand is calculated as a function of population density per age category in a particular outward code weighted by the level of medical assistance required.  

Since the city is built from scratch, the government has to build different kinds of facilities to accomodate people's needs. Therefore there is a restriction on the budget availability, number of hospitals built,and hospital capacity. 


### Scope

These different sectors are broken down by the outward section of the postal codes (i.e., the first three digits of the postal code before the space in the middle); within Montreal, there are a total of 116 outward codes.

The age groups are broken down into the four following groups: 0 – 14-year-olds, 15 – 64-year-olds, 65 – 84-year-olds, 85+ year olds.

# Import Packages

In [2]:
###Importing required library
import pandas as pd
import numpy as np
import gurobipy as gb
from gurobipy import GRB
from gurobipy import *

In [3]:
###file directory path for importing the dataset
import os
os.chdir(r'C:/Users/hp/OneDrive/Desktop/MMA/Fall 2022/MGSC 662 Decision Analytics/Assignment/Group assignment')

# Load and Analyze Required Data

In [4]:
###Datasets for coding
#population dataset
df = pd.read_csv("Population_Density_data.csv")
df_new = df[["0 to 14 years","15 to 64 years","65-84 years","85 years and over"]]
###postal code names
postal_code = df[["Postal Code"]]
####ditance between postal codes
#dist = [116*116] matrix
df2 = pd.read_excel("distance_matrix_int.xlsx")
df2 = df2.iloc[:,1:]

In [5]:
age = ["0-14yrs","15-64yrs","65-84yrs","85+yrs"]   ##age categories

med_assist = [0.6,0.3,0.7,0.8]  ###percentage of population that requires frequent medical asistance

pop_den = df_new.to_numpy()     ###population density matrix

Tot_pop = pop_den.sum(axis=1)   ##total population

postal_code = postal_code.to_numpy()    ##postalcode matrix

post_dist = df2.to_numpy()
post_dist = post_dist.astype(int)    ###distance between postal codes

###cost constraints
budget = 9000       ##Budget in million
FC_hos = 500         ##Fixed cost in million
VC_bed = 1           ##Variable cost in million

# Optimizing Model

### Decision Variables 

In [6]:
###Decision variables 
prob = gb.Model('Hospital Construction Optimization')
I = len(postal_code)
J = len(age)


###indicates where a hospital should be built in that postal code
X = prob.addVars(I, vtype = GRB.BINARY, name = [str(i) for i in postal_code])

###indicates the number of beds allocated to a hospital
B = prob.addVars(I, vtype = GRB.INTEGER, lb = 0, ub = 1000, name = ['Number of beds in ' + str(i) for i in postal_code])

##indicates which hospital doe people from location i access
Aux = prob.addVars(I,I,vtype = GRB.BINARY, name=["people from " +str(i) + "visits" +str(j) for i in postal_code for j in postal_code ])

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-15


Maximizing the service accessibility of hospitals for the most required age group based on the level of medical assistance required age-wise  

$$\max \sum_{i,j} (X_i * P_{i,j} * A_j) +(Aux_i * P_{i,j} * A_j) $$


In [7]:
###objective function
###maximize the medical assistanceship received to the population 

prob.setObjective(sum((X[i] * pop_den [i,j] * med_assist[j]+(pop_den [i,j] * Aux[i,j] * med_assist[j])) for i in range(I) for j in range (J)), GRB.MAXIMIZE)

### Constraints

The model has 350 constraints

In [53]:
#Aux[i][j]:-Aux[people location][hospital location]

#Budget constraint
prob.addConstr(sum(X[i] * FC_hos + B[i] * VC_bed for i in range(I)) <= budget,name='Budget') 

#At least 12 hospitals in the city
prob.addConstr(sum(X[i] for i in range(I))>=12, name='Max number of hospital required')    

##the variable cost should be included only if the hospital is present in that location 
for i in range(I):
    prob.addConstr(B[i] <=(100000*X[i]),name="Beds only if hopsital is present")
    

##people from every postal code is assignmented to a hospital 
##people from location i visits only one hospital in location j (if present)
for i in range(I):  
    prob.addConstr(sum(Aux[i,j] for j in range(I))==1,name='Aux allocate')
    
for i in range(I):  
    prob.addConstr(sum(Aux[i,j]*post_dist[i][j] for j in range(I))<=10,name='The allocated hospital should be less than 10kms far')
    
###number of bed per hospital proportional to population 

##population from the same outward code is given more priorit
#0.2% of the total population of that postal code + 0.02% of the total population of postal codes that will use this hospital
for j in range(I):
    prob.addConstr(quicksum(0.0002*Tot_pop[i]*Aux[i,j] for i in range(I)) + (X[j]*0.002*Tot_pop[j])<=B[j],name="Beds allocated")             
               


Number of DV and constraints

In [54]:
####model optimization
prob.Params.NonConvex = 2
prob.optimize()
#prob.display()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1398 rows, 13688 columns and 123192 nonzeros
Model fingerprint: 0x1ee5f744
Variable types: 0 continuous, 13688 integer (13572 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+05]
  Objective range  [4e+00, 2e+04]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 9e+03]

Loaded MIP start from previous solve with objective 261641

Presolve removed 1050 rows and 9566 columns
Presolve time: 0.08s
Presolved: 348 rows, 4122 columns, 8592 nonzeros
Variable types: 0 continuous, 4122 integer (4007 binary)

Root relaxation: objective 2.814624e+05, 1037 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 281462.401    0   42 261641.000 281462.401  7.58%     -    0s
    

# Results

In [59]:
##results
count = 0
loc = []
code = []
beds = []
o=0
for v in prob.getVars():
    o +=1
    if v.x==1:
        count += 1
        print(v.varName, '=', v.x)
        if count<=15:
            code.append(v.varName)
        else:
            loc.append(v.varName)
    if  v.x>1:
        print(v.varName, '=', v.x)
        beds.append(int(v.x))
len(loc)
o

['H1B'] = 1.0
['H1E'] = 1.0
['H1G'] = 1.0
['H1H'] = 1.0
['H1K'] = 1.0
['H1Z'] = 1.0
['H4E'] = 1.0
['H4L'] = 1.0
['H8N'] = 1.0
['H7C'] = 1.0
['H7L'] = 1.0
['H7N'] = 1.0
['H7W'] = 1.0
['H9H'] = 1.0
['H9W'] = 1.0
Number of beds in ['H1B'] = 89.0
Number of beds in ['H1E'] = 127.0
Number of beds in ['H1G'] = 98.0
Number of beds in ['H1H'] = 165.0
Number of beds in ['H1K'] = 82.0
Number of beds in ['H1Z'] = 99.0
Number of beds in ['H4E'] = 115.0
Number of beds in ['H4L'] = 134.0
Number of beds in ['H8N'] = 80.0
Number of beds in ['H7C'] = 27.0
Number of beds in ['H7L'] = 123.0
Number of beds in ['H7N'] = 90.0
Number of beds in ['H7W'] = 103.0
Number of beds in ['H9H'] = 93.0
Number of beds in ['H9W'] = 72.0
people from ['H1A']visits['H1B'] = 1.0
people from ['H1B']visits['H1B'] = 1.0
people from ['H1C']visits['H1B'] = 1.0
people from ['H1E']visits['H1B'] = 1.0
people from ['H1G']visits['H1E'] = 1.0
people from ['H1H']visits['H1E'] = 1.0
people from ['H1J']visits['H1B'] = 1.0
people from ['H1

13688

#### There are a total of 15 hospitals that get constructed.

### Binding and Non-Binding Constraint

In [60]:
count = 0
for c in prob.getConstrs():
    if c.Slack > 1e-6:
        print(c.ConstrName, ": The slack is ", round(c.Slack))
        count += 1
    
count

Budget : The slack is  3
R3 : The slack is  99911
R5 : The slack is  99873
R6 : The slack is  99902
R7 : The slack is  99835
R9 : The slack is  99918
R21 : The slack is  99901
R65 : The slack is  99885
R70 : The slack is  99866
R79 : The slack is  99920
R88 : The slack is  99973
R94 : The slack is  99877
R96 : The slack is  99910
R102 : The slack is  99897
R110 : The slack is  99907
R116 : The slack is  99928
Aux allocate : The slack is  5
Aux allocate : The slack is  10
Aux allocate : The slack is  3
Aux allocate : The slack is  1
Aux allocate : The slack is  5
Aux allocate : The slack is  2
Aux allocate : The slack is  3
Aux allocate : The slack is  4
Aux allocate : The slack is  1
Aux allocate : The slack is  1
Aux allocate : The slack is  3
Aux allocate : The slack is  1
Aux allocate : The slack is  1
Aux allocate : The slack is  5
Aux allocate : The slack is  2
Aux allocate : The slack is  2
Aux allocate : The slack is  2
Aux allocate : The slack is  3
Aux allocate : The slack is 

360

### Location of the Hospitals

In [49]:
###Data preprocessing for visualization
import re
final = []
final1 = []
final2 = []
result =''
for i in range(15):
    sampleStr = code[i]
    mk1 = sampleStr.find("['")+2
    mk2 = sampleStr.find("']", mk1)
    result = sampleStr[ mk1 : mk2 ]
    final.append(result)                  ###contains the postal codes of all the locations where hospital is to be built 
    
soln ={'Outward codes':final,"beds":beds} 

sol = pd.DataFrame(data=soln)

for i in range(len(loc)):
    sampleStr = loc[i]
    mk1 = sampleStr.find("['")+2
    mk2 = sampleStr.find("']visits", mk1)
    result1 = sampleStr[ mk1 : mk2 ]
    final1.append(result1) 
    
    sampleStr2 = loc[i]
    mk12 = sampleStr2.find("visits['")+8
    mk22 = sampleStr2.find("'] =1.0", mk12)-1
    result2 = sampleStr[ mk12 : mk22 ]
    final2.append(result2)     

so ={'Outward codes':final1,"Hospital":final2}                

In [50]:
import pgeocode
nomi = pgeocode.Nominatim('ca')
df = nomi.query_postal_code(final)
df["No. of beds"] = beds
print("\033[1m" + "Outward codes that will have hospitals" + "\033[0m")
df[["postal_code","place_name",'No. of beds']]

Outward codes that will have hospitals


,postal_code,place_name,No. of beds
0,H1B,Montreal East,89
1,H1E,Rivière-Des-Prairies Southwest,127
2,H1G,Montreal North North,98
3,H1H,Montreal North South,165
4,H1K,Anjou East,82
5,H1Z,Saint-Michel West,99
6,H4E,Ville Émard,115
7,H4L,Saint-Laurent Inner Northeast,134
8,H8N,LaSalle Northwest,80
9,H7C,Saint-Vincent-de-Paul,27


In [51]:
print("\033[1m" + "People in Outward codes visit its respective hospitals" + "\033[0m")
locat= pd.DataFrame(data=so) 
locat.head(20)

People in Outward codes visit its respective hospitals


,Outward codes,Hospital
0,H1A,H1B
1,H1B,H1B
2,H1C,H1B
3,H1E,H1B
4,H1G,H1E
5,H1H,H1E
6,H1J,H1B
7,H1K,H1B
8,H1L,H1B
9,H1M,H1B


In [22]:
##extract montreal's base map location coordinates
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="location_details")
location = geolocator.geocode("Montreal,Canada")

In [23]:
###create a map to indicate the choosen hospital locations
import folium
map=folium.Map(location=(location.latitude, location.longitude))
for i in range(11):
    map.add_child(folium.Marker(location=[df['latitude'][i],df['longitude'][i]],popup=df['place_name'][i],icon=folium.Icon(color='green')))
map


In [24]:
###calculating the distance between various postal codes
import geopy.distance
dist=[]
cor_1 = []
cor_2 = []
for i in range(12):
    for j in range(i+1):
        if(i!=j):
            coords_1 = (df['latitude'][i],df['longitude'][i])
            coords_2 = (df['latitude'][j],df['longitude'][j])
            dist.append(geopy.distance.geodesic(coords_1, coords_2).km)
            cor_1.append(df["postal_code"][i])
            cor_2.append(df["postal_code"][j])
            
distance = {"Hospital location 1":cor_1,"Hospital location 2":cor_2,"distance":dist}
sol = pd.DataFrame(data=distance)

print("\033[1m" + "Top Closest hospitals" + "\033[0m")
sol.sort_values(['distance']).head(10)

Top Closest hospitals


,Hospital location 1,Hospital location 2,distance
38,H7C,H1G,2.287803
34,H8N,H4E,2.577094
13,H1Z,H1H,2.628785
5,H1H,H1G,2.715919
39,H7C,H1H,3.095864
2,H1G,H1E,3.871416
62,H7N,H4L,3.887823
6,H1K,H1B,3.966062
7,H1K,H1E,3.967563
12,H1Z,H1G,4.613107


In [25]:
print("\033[1m" + "Top farthest hospitals" + "\033[0m")
sol.sort_values(['distance'],ascending=False).head(10)

Top farthest hospitals


,Hospital location 1,Hospital location 2,distance
53,H7L,H8N,23.517940
28,H8N,H1B,23.327924
51,H7L,H4E,23.055066
29,H8N,H1E,21.999875
45,H7L,H1B,21.512511
15,H4E,H1B,20.958411
16,H4E,H1E,19.992394
44,H7C,H8N,19.989674
32,H8N,H1K,19.946389
30,H8N,H1G,19.216482


In [52]:
##extract montreal's base map location coordinates
import pgeocode
nomi = pgeocode.Nominatim('ca')
df2 = nomi.query_postal_code(final1)

In [35]:
###create a map to indicate the choosen hospital locations
import folium
map2=folium.Map(location=(location.latitude, location.longitude))
for i in range(116):
    map2.add_child(folium.Marker(location=[df['latitude'][df.postal_code=="H1B"],df['longitude'][df.postal_code=="H1B"]],popup=df['place_name'][df.postal_code=="H1B"],icon=folium.Icon(color='blue')))
    if final2[i] == "H1B":
        map2.add_child(folium.Marker(location=[df2['latitude'][i],df2['longitude'][i]],popup=df2['place_name'][i],icon=folium.Icon(color='green')))
map2


In [36]:
###create a map to indicate the choosen hospital locations
import folium
map2=folium.Map(location=(location.latitude, location.longitude))
for i in range(116):
    map2.add_child(folium.Marker(location=[df['latitude'][df.postal_code=="H9H"],df['longitude'][df.postal_code=="H9H"]],popup=df['place_name'][df.postal_code=="H9H"],icon=folium.Icon(color='blue')))
    if final2[i] == "H9H":
        map2.add_child(folium.Marker(location=[df2['latitude'][i],df2['longitude'][i]],popup=df2['place_name'][i],icon=folium.Icon(color='green')))
map2
